In [1]:
! pip install unsloth transformers datasets xformers huggingface_hub torch torchvision torchaudio bitsandbytes peft trl tiktoken wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 40.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 35.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 72.7 MB/s et

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# WandB

init(): https://docs.wandb.ai/ref/python/init/

In [3]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: laravelshubham (laravelshubham-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
run = wandb.init(
    project='Fine-tuning Gemma-3-4B on FinQA Reasoning Dataset', 
    job_type="training", 
    anonymous="allow"
)

In [5]:
from unsloth import FastModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothGKDTrainer: No module named 'UnslothGKDTrainer'. Using tempfile instead!


In [6]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now
)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.5 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

### Now add LoRA adapters so we only need to update a small amount of parameters!

In [7]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


## System Prompt

In [37]:
system_prompt = """
There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.
Write a response that appropriately completes the request.

Before providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.

### Question:
{}

### Response:
<think>
{}
</think>
{}
""".strip()

#### Why to add EOS

Yes, for models like Gemma (and other decoder-only models like LLaMA, Falcon, Mistral), it’s essential to add the End-of-Sequence (EOS) token explicitly in certain scenarios — especially when training, fine-tuning, or prompting for generation.

🧠 Why You Must Add the EOS Token in Gemma
1️⃣ Gemma is a decoder-only causal language model
It learns to predict the next token, one token at a time, based on previous context. It needs to know when a sequence ends — and that’s what the eos_token_id marks.

2️⃣ During training and generation, EOS serves two roles:
Purpose	Effect
In training	Tells the model that prediction should stop at a certain point, helps it learn boundaries between prompt and response.
In generation	Model will stop decoding when it outputs the eos_token_id, saving time and preventing infinite loops.

⚙️ How to Know When You Must Add EOS
There are 3 clear signs:

✅ 1. Tokenizer config lacks automatic EOS addition

✅ 2. Gemma model docs say it
In the Gemma docs, it clearly states:

❗️ "To get the model to stop generating, you must ensure the input includes the eos_token_id, as the model was trained to stop generation upon seeing it."

✅ 3. generate() doesn’t stop unless EOS is reached
If your .generate() keeps going too long or hits the max_new_tokens limit every time, it's a sign that:

The model doesn’t know when to stop

It never emitted eos_token_id

In [13]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

In [28]:
def test_formatting_prompts_func(examples):
    inputs = examples["Open-ended Verifiable Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []

    zipped = zip(inputs,complex_cots,outputs)
    print("Check zipped: ", zipped)
    print("=======================")
    
    # For each triplet (input, cot, output), it formats them into a single string using system_prompt.format().
    for input, cot, output in zip(inputs,complex_cots,outputs):
        print("zipped:  ", input,cot,output)
        print("=======================")
        text = system_prompt.format(input,cot,output) + EOS_TOKEN
        print("Formatted Value: ",texts)
        print("=======================")
        texts.append(text)
        print("========DONE===============")
    
    return {
        "text": texts,
    }

In [38]:
def formatting_prompts_func(examples):
    inputs = examples["Open-ended Verifiable Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []

    
    # For each triplet (input, cot, output), it formats them into a single string using system_prompt.format().
    for input, cot, output in zip(inputs,complex_cots,outputs):
        text = system_prompt.format(input,cot,output) + EOS_TOKEN
        texts.append(text)
        
    
    return {
        "text": texts,
    }

## Pull and Format the Dataset

In [15]:
from datasets import load_dataset
dataset = load_dataset("TheFinAI/Fino1_Reasoning_Path_FinQA", split = "train[0:500]",trust_remote_code=True)

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5499 [00:00<?, ? examples/s]

In [16]:
len(dataset)

500

In [22]:
test_ip = dataset[0]
test_ip

{'Open-ended Verifiable Question': 'Please answer the given financial question based on the context.\nContext: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .\n|2017|$ 10509|\n|2018|9346|\n|2019|9240|\n|2020|7201|\n|2021|5318|\n|2022 and thereafter|16756|\n|amortization expense of intangible assets|$ 58370|\nat december 31 , 2016 , 2015 and 2014 , the company determined that its goodwill and indefinite- lived intangible assets were not impaired . 5 . credit facility and other long term debt credit facility the company is party to a credit agreement that provides revolving commitments for up to $ 1.25 billion of borrowings , as well as term loan commitments , in each case maturing in 

In [23]:
type(test_ip)

dict

In [31]:
# test_result = test_formatting_prompts_func(test_ip)

### Map dataset in batches

The dataset.map() function applies formatting_prompts_func to all batches in the dataset.

The original dataset is transformed into a new dataset where each example is replaced by the formatted text.

The new dataset will have a column named "text" containing the formatted prompts (instead of the original separate columns).

In [39]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [40]:
dataset[0]

{'Open-ended Verifiable Question': 'Please answer the given financial question based on the context.\nContext: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .\n|2017|$ 10509|\n|2018|9346|\n|2019|9240|\n|2020|7201|\n|2021|5318|\n|2022 and thereafter|16756|\n|amortization expense of intangible assets|$ 58370|\nat december 31 , 2016 , 2015 and 2014 , the company determined that its goodwill and indefinite- lived intangible assets were not impaired . 5 . credit facility and other long term debt credit facility the company is party to a credit agreement that provides revolving commitments for up to $ 1.25 billion of borrowings , as well as term loan commitments , in each case maturing in 

In [42]:
dataset[0]['text']

"There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.\nWrite a response that appropriately completes the request.\n\nBefore providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.\n\n### Question:\nPlease answer the given financial question based on the context.\nContext: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .\n|2017|$ 10509|\n|2018|9346|\n|2019|9240|\n|2020|7201|\n|2021|5318|\n|2022 and thereafter|16756|\n|amortization expense of intangible assets|$ 58370|\nat december 31 , 2016 

# Train Model

In [ ]:
# from trl import SFTTrainer, SFTConfig

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
    
#     args = SFTConfig(
#         output_dir="outputs",
#         dataset_text_field = "text",
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4, # Use GA to mimic batch size!
#         warmup_steps = 5,
#         # num_train_epochs = 1, # Set this for 1 full training run.
#         max_steps = 30,
#         max_seq_length=1024,
#         learning_rate = 2e-3, # Reduce to 2e-5 for long training runs
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         report_to = "none", # Use this for WandB etc
#         dataset_num_proc=2,
#     ),
# )

### unsloth CLI: 
https://github.com/unslothai/unsloth/blob/main/unsloth-cli.py#L121

In [51]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [52]:
# Configure training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    max_steps=300,
    learning_rate=2e-3,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to = "none", # Use this for WandB = "wandb", etc
)

In [53]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    dataset_num_proc=2,
    packing=False, #  Whether to pack multiple sequences into a fixed-length format. Uses max_length to define sequence length.
    args=training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [54]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 6000 Ada Generation. Max memory = 47.5 GB.
4.955 GB of memory reserved.


In [55]:
trainer_stats = trainer.train()
# To resume a training run, set trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 2 | Total steps = 300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Step,Training Loss
10,1.132300
20,1.146000
30,1.328100
40,1.555700
50,1.408400
60,1.422700
70,1.536300
80,1.449900
90,1.355100
100,1.466200


In [56]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

218.6411 seconds used for training.
3.64 minutes used for training.
Peak reserved memory = 4.955 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 10.432 %.
Peak reserved memory for training % of max memory = 0.0 %.


# Infer on FT Adapters

In [59]:
FastModel.for_inference(model)  # Unsloth has 2x faster inference!

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(4096, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-26): 27 x SiglipEncoderLayer(
                (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                (self_attn): SiglipAttention(
                  (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
                

In [83]:
prompt = """
There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.
Write a response that appropriately completes the request.

Before providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.

### Question:
{}

### Response:
<think>
{}

""".strip()

In [77]:
question = """
Please answer the given financial question based on the context.
Context: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .
|2017|$ 10509|
|2018|9346|
|2019|9240|
|2020|7201|
|2021|5318|
|2022 and thereafter|16756|
|amortization expense of intangible assets|$ 58370|
at december 31 , 2016 , 2015 and 2014 , the company determined that its goodwill and indefinite- lived intangible assets were not impaired . 5 . credit facility and other long term debt credit facility the company is party to a credit agreement that provides revolving commitments for up to $ 1.25 billion of borrowings , as well as term loan commitments , in each case maturing in january 2021 . as of december 31 , 2016 there was no outstanding balance under the revolving credit facility and $ 186.3 million of term loan borrowings remained outstanding . at the company 2019s request and the lender 2019s consent , revolving and or term loan borrowings may be increased by up to $ 300.0 million in aggregate , subject to certain conditions as set forth in the credit agreement , as amended . incremental borrowings are uncommitted and the availability thereof , will depend on market conditions at the time the company seeks to incur such borrowings . the borrowings under the revolving credit facility have maturities of less than one year . up to $ 50.0 million of the facility may be used for the issuance of letters of credit . there were $ 2.6 million of letters of credit outstanding as of december 31 , 2016 . the credit agreement contains negative covenants that , subject to significant exceptions , limit the ability of the company and its subsidiaries to , among other things , incur additional indebtedness , make restricted payments , pledge their assets as security , make investments , loans , advances , guarantees and acquisitions , undergo fundamental changes and enter into transactions with affiliates . the company is also required to maintain a ratio of consolidated ebitda , as defined in the credit agreement , to consolidated interest expense of not less than 3.50 to 1.00 and is not permitted to allow the ratio of consolidated total indebtedness to consolidated ebitda to be greater than 3.25 to 1.00 ( 201cconsolidated leverage ratio 201d ) . as of december 31 , 2016 , the company was in compliance with these ratios . in addition , the credit agreement contains events of default that are customary for a facility of this nature , and includes a cross default provision whereby an event of default under other material indebtedness , as defined in the credit agreement , will be considered an event of default under the credit agreement . borrowings under the credit agreement bear interest at a rate per annum equal to , at the company 2019s option , either ( a ) an alternate base rate , or ( b ) a rate based on the rates applicable for deposits in the interbank market for u.s . dollars or the applicable currency in which the loans are made ( 201cadjusted libor 201d ) , plus in each case an applicable margin . the applicable margin for loans will .
Question: what portion of the estimated amortization expense will be recognized in 2017?
Answer:
""".strip()

In [69]:
prompt_format = prompt.format(question, "")

In [70]:
inputs = tokenizer([prompt_format], return_tensors="pt").to("cuda")

In [71]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1024,
    use_cache=True,
)

In [72]:
response = tokenizer.batch_decode(outputs)

In [73]:
response

["<bos>There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.\nWrite a response that appropriately completes the request.\n\nBefore providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.\n\n### Question:\nPlease answer the given financial question based on the context.\nContext: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .\n|2017|$ 10509|\n|2018|9346|\n|2019|9240|\n|2020|7201|\n|2021|5318|\n|2022 and thereafter|16756|\n|amortization expense of intangible assets|$ 58370|\nat december 31 ,

In [87]:
question = "Given two integers `x` and `y`, can you write a python function that returns the greatest common divisor of `x` and `y`?"

prompt_format = prompt.format(question, "")

inputs = tokenizer([prompt_format], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1500,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [88]:
response

['<bos>There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.\nWrite a response that appropriately completes the request.\n\nBefore providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.\n\n### Question:\nGiven two integers `x` and `y`, can you write a python function that returns the greatest common divisor of `x` and `y`?\n\n### Response:\n<think>\nOkay, let\'s figure out a way to find the greatest common divisor (GCD) of two numbers in Python. We\'re looking at numbers `x` and `y` that we want to find the GCD of.\n\nFirst, I can think about the traditional way to find the GCD. This is the method of dividing, assuming both numbers are positive integers. We start by dividing `x` by `y` and taking the remainder, which I\'ll store in a variable called `r`.\n\nNext, I\'ll take `y` and divide it

In [90]:
print(response[0].split("### Response:")[1])


<think>
Okay, let's figure out a way to find the greatest common divisor (GCD) of two numbers in Python. We're looking at numbers `x` and `y` that we want to find the GCD of.

First, I can think about the traditional way to find the GCD. This is the method of dividing, assuming both numbers are positive integers. We start by dividing `x` by `y` and taking the remainder, which I'll store in a variable called `r`.

Next, I'll take `y` and divide it by the remainder `r`. The new remainder will be stored in a variable called `temp_r`.

Then, I'll take the previous remainder, `r`, and divide it by the new remainder, `temp_r`. This creates another remainder, `temp_r_2`, and I'll store it in a variable called `temp_r_2`.

Let's keep going with this process. Each time I divide, I'll update the remainder until the remainder is 0. In this case, when `temp_r_2` becomes 0, the previous remainder, `temp_r_1`, is the GCD.

Oh, so far so good! It looks like the remainder is indeed the GCD whenever w

In [93]:
question = "Given an integer number 'n', write python code to give me fibonacii number?"

prompt_format = prompt.format(question, "")

inputs = tokenizer([prompt_format], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2000,
    use_cache=True,
    early_stopping=True,
)

response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])


<think>
Alright, so let's try to figure this out. I need to take an integer, which I'll call 'n', and then find the corresponding Fibonacci number.

First, I need to understand what the Fibonacci numbers are. These numbers are part of a sequence where each number in the sequence is the sum of the previous two numbers. So, if I take a starting number, let's say 0, and then add the previous number, which is also 0, to get the first number in the sequence, then 0 plus 0 gives me 0. If I start with 1 and add the previous number, 1, I get 1. That's the sequence: 0, 1, 1, 2, 3, 5, and so on.

Now, how do I find the n-th Fibonacci number? To do that, I can use a simple recursive approach. So, let's set up a function that takes 'n' as an argument. The function checks if 'n' is 0. If it is, we return 0, otherwise if 'n' is 1, we return 1. If 'n' is greater than 1, we call the function again to find the (n-1)-th Fibonacci number. Then, we add the (n-1)-th Fibonacci number to the (n-2)-th Fibona

# Save and Push to HUB

In [94]:
# Save Locally - Adapters Only
new_model_local = "Gemma-3-4B-Fin-QA-Reasoning"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

['Gemma-3-4B-Fin-QA-Reasoning/processor_config.json']

In [95]:
# Save Locally - Merged Model
new_ft_model = "Rathsam/Gemma-3-4B-Financial-QA-Reasoning"

if True:
    model.save_pretrained_merged(new_ft_model, tokenizer)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [01:27<01:27, 87.31s/it]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [02:27<00:00, 73.55s/it]


In [109]:
# new_ft_model = "Rathsam/Gemma-3-4B-Fin-QA-Reasoning"

# if True: # Change to True to upload finetune
#     model.push_to_hub_merged(
#         new_ft_model, tokenizer,
#         token = ""
#     )

# Inference on merged model

In [98]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "Rathsam/Gemma-3-4B-Fin-QA-Reasoning",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.5 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at Rathsam/Gemma-3-4B-Fin-QA-Reasoning were not used when initializing Gemma3ForCausalLM: ['multi_modal_projector.mm_input_projection_weight', 'multi_modal_projector.mm_soft_emb_norm.weight', 'vision_tower.vision_model.embeddings.patch_embedding.bias', 'vision_tower.vision_model.embeddings.patch_embedding.weight', 'vision_tower.vision_model.embeddings.position_embedding.weight', 'vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'vision_tower.vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.bias', 'vision_tower.vis

In [101]:
question = "Given an integer number 'n', write python code to give me fibonacii number?"

prompt_format = prompt.format(question, "")

inputs = tokenizer([prompt_format], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
    early_stopping=True,
)

response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])


<think>
Alright, let's figure out how to find the Fibonacci number for a given number 'n'.

First, I need to understand what the Fibonacci sequence is all about. It starts with 0 and 1, and each subsequent number is the sum of the two preceding ones. So, if I have 0 and 1, the next number is 1, then 1 plus 1 is 2, and so on.

Now, let's think about how to find the Fibonacci number for a specific 'n'. I can do this by creating a function that takes 'n' as input and returns the Fibonacci number for that 'n'.

Let's start by defining the base cases. If n is 0, the Fibonacci number is 0. If n is 1, the Fibonacci number is 1. Got it.

Next, I need to define a recursive function to calculate the Fibonacci number. This function should take 'n' as input and return the Fibonacci number for 'n'.

Let's write this down:
```python
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

This function works by

In [100]:
# Save the fine-tuned model
wandb.finish()